In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## Cargar Datos

In [9]:
#!git clone "https://github.com/cienciadatos/niche_fraudsters.git"
#!unzip "niche_fraudsters/data_fraude.zip"
#!unzip "niche_fraudsters/df_one_hot.zip"

In [1]:
DATA_TRAIN  = "X_train.csv"
DATA_Y      = "Y_train.csv"
DATA_TEST   = "X_test.csv"

In [2]:
DF_OH_TRAIN = "df_one_hot_train.csv"
DF_OH_TEST  = "df_one_hot_test.csv"

## Importar Librerías

In [5]:
# Librerías Básicas
# ==============================================================================
import pandas as pd
import numpy as np

# Eliminar warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

## Funciones Auxiliares

In [6]:
def limpiar_txt_items(df):
    """Elimina espacios, comas, caracteres raros de las variables itemX con el
    fin de unificar valores y eliminar duplicados.
    """
    df.iloc[:,1:25] = df.iloc[:,1:25].replace(r'[^0-9a-zA-Z ]', '', 
                        regex=True).replace(r'\s+',' ',regex=True)
    return df

def data_to_pandas(DATA_TRAIN, DATA_Y):
    """Generamos dataset de pandas con los csv y unimos el X_train con el 
    Y_train, que contiene la etiqueta de clase (fraud_flag). Devuelve
    el dataset en pandas.DataFrame.
    """
    x_train = pd.read_csv(DATA_TRAIN, sep=",")
    y_train = pd.read_csv(DATA_Y, sep=",")
    y_train = y_train.drop('index', axis=1)
    df = x_train.merge(y_train)
    df = limpiar_txt_items(df)
    return df

def lista_transacciones_vcalc(df):
    """Crear una lista de diccionarios para cada fila del dataframe.
    Devuelve una lista de transacciones.
    """
    list_trans = []
    for index, row in df.iterrows():
        precio = []
        nb_pp = []
        nb_items = int(row[f'Nb_of_items'])
        if nb_items > 24: nb_items = 24 # los que tienen más de 24 se dejan a 24.
        fraud = row['fraud_flag']
        for i in range(1,nb_items+1):
            precio.append(row[f'cash_price{i}'])
            nb_pp.append(int(row[f'Nbr_of_prod_purchas{i}']))
        transaccion = {
            'id': row['ID'],
            'num_items_dist': nb_items,
            'sum_items_total': sum(nb_pp),
            'precio_total': sum(precio),
            'max_num_prods_item': max(nb_pp),
            'precio_max_item': max(precio),
            'dif_precio_min_max':abs(max(precio)-min(precio)),
            'precio_unit_item_mas_comprado': precio[nb_pp.index(max(nb_pp))]/max(nb_pp),
            'apple_prod': row['apple_prod'],
            'computers_prod': row['computers_prod'],
            'warranty_prod': row['warranty_prod'],
            'fulfilment_prod': row['fulfilment_prod'],
            'fraud_flag': fraud
        }
        list_trans.append(transaccion)
    return list_trans

def lista_transacciones_vcalc_test(df):
    """Crear una lista de diccionarios para cada fila del dataframe.
    Devuelve una lista de transacciones.
    """
    list_trans = []
    for index, row in df.iterrows():
        # Crear una lista de tuplas para cada artículo y su precio
        precio = []
        nb_pp = []
        nb_items = int(row[f'Nb_of_items'])
        if nb_items > 24: nb_items = 24 # los que tienen más de 24 se dejan a 24.
        for i in range(1,nb_items+1):
            precio.append(row[f'cash_price{i}'])
            nb_pp.append(int(row[f'Nbr_of_prod_purchas{i}']))
        transaccion = {
            'id': row['ID'],
            'num_items_dist': nb_items,
            'sum_items_total': sum(nb_pp),
            'precio_total': sum(precio),
            'max_num_prods_item': max(nb_pp),
            'precio_max_item': max(precio),
            'dif_precio_min_max':abs(max(precio)-min(precio)),
            'precio_unit_item_mas_comprado': precio[nb_pp.index(max(nb_pp))]/max(nb_pp),
            'apple_prod': row['apple_prod'],
            'computers_prod': row['computers_prod'],
            'warranty_prod': row['warranty_prod'],
            'fulfilment_prod': row['fulfilment_prod']
        }
        list_trans.append(transaccion)
    return list_trans

def data_to_csv(df_one_hot, nombre):
    """Genera un csv con el dataset one_hot_encoder."""
    df_one_hot.to_csv(nombre+'.csv',sep=",",header=True,index=False)
    
def genera_prod(df_train,prod,col_ini,col_fin):
    dx = df_train
    d_model = dx.iloc[:,col_ini:col_fin+1]
    d_model['ID'] = dx.ID
    d_model['Nb_items'] = dx.Nb_of_items
    list_prod = []
    for i,row in d_model.iterrows():
        num_items = int(row['Nb_items'])
        if num_items > 24: num_items = 24
        fila = row.iloc[:num_items]
        if not all(fila.isnull()):
            if any(x for x in fila.values if prod.lower() in x.lower()) == True:
                list_prod.append((row['ID'],True))
                continue
        list_prod.append((row['ID'],False))
    dx[prod+'_prod'] = pd.DataFrame(list_prod, 
                        columns=['ID',prod+'_prod']).drop('ID',axis=1)
    return dx

def prec_mayor(x):
    if x <= 600:
        return False
    else:
        return True

def nb_max_menor_8(x):
    if x <= 8:
        return True
    else:
        return False
    
def generar_probf(DF_OH_TRAIN, DF_OH_TEST, df_vcalc_train, df_vcalc_test):

    df = pd.read_csv(DF_OH_TRAIN)
    df = df.set_index('id')
    dt = pd.read_csv(DF_OH_TEST)
    dt = dt.set_index('id')
    
    dt = dt.reindex(columns = df.columns, fill_value=0)
    df = df.drop('Nb_of_items', axis=1)
    dt = dt.drop('Nb_of_items', axis=1)

    # Identificar los patrones de duplicados sin contar la columna 'fraud_flag'
    columnas = df.columns.tolist()
    columnas.remove('fraud_flag')
    patrones_duplicados = df.duplicated(subset=columnas, keep=False)

    # Filtrar el DataFrame original para obtener solo las filas duplicadas
    filas_duplicadas = df[patrones_duplicados]

    # Obtener los patrones únicos sin la columna 'fraud_flag'
    patrones_unicos = filas_duplicadas.drop_duplicates(subset=columnas)

    # TRAIN ##########
    df['probf'] = 0.5
    
    # Recorrer los patrones únicos y actualizar la columna 'probf' en todas las 
    # filas que cumplan ese patrón
    cont_nf, cont_f, cont_mix = 0, 0, 0
    for _, patron in patrones_unicos.iterrows():
        mask = (df[columnas] == patron[columnas]).all(axis=1)
        support1 = (df.loc[mask, 'fraud_flag'] == 1).sum() / (df.loc[mask,'fraud_flag'].notna().sum())
        if support1 == 0: cont_nf+=1
        elif support1 == 1: cont_f+=1
        else: cont_mix+=1
        df.loc[mask, 'probf'] = support1
    
    #print(cont_nf, cont_f, cont_mix)
    
    df['probf'] = df['probf'].fillna(0.5)
    df_vcalc_train['probf'] = df.reindex(df_vcalc_train.set_index('id').index)['probf'].values
    
    # TEST ##########
    # Inicializar la columna 'probf' en el DataFrame de prueba
    df_test['probf'] = 0.5

    # Recorrer los patrones únicos y actualizar la columna 'probf' 
    #en el DataFrame de prueba
    for _, patron in patrones_unicos.iterrows():
        # Crear una máscara booleana para identificar las filas en el DataFrame de prueba 
        #que cumplen el patrón
        mask = (dt[columnas] == patron[columnas]).all(axis=1)
        # Obtener el valor de 'probf' del DataFrame original para el primer valor no nulo 
        #en las filas que cumplen el patrón
        probf = df.loc[df[columnas].eq(patron[columnas]).all(axis=1), 
                       'probf'].dropna().iloc[0]
        # Actualizar el valor de 'probf' en el DataFrame de prueba
        dt.loc[mask, 'probf'] = probf

    # Llenar los valores faltantes en la columna 'probf' del DataFrame de prueba con 0.5
    dt['probf'] = dt['probf'].fillna(0.5)
    df_vcalc_test['probf'] = dt.reindex(df_vcalc_test.set_index('id').index)['probf'].values
    
    return df_vcalc_train, df_vcalc_test

## Crear Dataset Final (Fraud 3)

Vamos a modificar el dataset "Fraud 2" y ampliarlo, apostando más por el precio y el nº de productos. Las variables que se crearán son las siguientes:

`id (int)`: Id de la transacción. \
`num_items_dist (int)`: Nº de productos distintos en la transacción. \
`sum_items_total (int)`: Nº de productos en total. \
`precio_total (float)`: Precio total de la transacción. \
`max_num_prods_item (int)`: Nº máximo de productos iguales en una transacción. \
`precio_max_item (float)`: Precio máximo de un producto en una transacción. \
`dif_precio_min_max (float)`: Diferencia entre precio máximo y mínimo. \
`precio_unit_item_mas_comprado (float)`: Precio unitario del item más comprado. \
`precio_mayor_600 (bool)`: False si precio_total <= 600 y True si es > de 600. \
`nb_max_items_menor_8 (bool)`: False si max_num_prods_item > 8 y True si <= 8. \
`nb_items_mayor_20 (bool)`: False si sum_items_total < 20 y True si >= 20. \
`apple_prod (bool)`: False si no incluye apple. True si lo incluye. \
`computer_prod (bool)`: False si no incluye computer. True si lo incluye. \
**`warranty_prod (bool)`: False si no incluye warranty. True si lo incluye.** \
**`fulfilment_prod`:  False si no incluye fulfilment. True si lo incluye.** \
`probf (float)`: Probabilidad de fraude calculada con el confidence. \
`fraud_flag (bool)`: Etiqueta de fraude. 0 si es no fraude y 1 si es fraude.

In [7]:
# Inicializa los datasets y los prepara
# ==============================================================================
df_train = data_to_pandas(DATA_TRAIN,DATA_Y)
df_test = pd.read_csv(DATA_TEST)

In [8]:
# Genera columna Apple Prod
# ==============================================================================
df_train = genera_prod(df_train,'apple',73,96)
df_test = genera_prod(df_test,'apple',73,96)

In [9]:
# Genera columna Computers Prod
# ==============================================================================
df_train = genera_prod(df_train,'computers',1,24)
df_test = genera_prod(df_test,'computers',1,24)

In [10]:
# Genera columna Warranty Prod
# ==============================================================================
df_train = genera_prod(df_train,'warranty',1,24)
df_test = genera_prod(df_test,'warranty',1,24)

In [11]:
# Genera columna Fulfilment Prod
# ==============================================================================
df_train = genera_prod(df_train,'fulfilment',1,24)
df_test = genera_prod(df_test,'fulfilment',1,24)

In [16]:
# Genera dataset con Variables Calculadas
# ==============================================================================
df_final_train = pd.DataFrame(lista_transacciones_vcalc(df_train))
df_final_test  = pd.DataFrame(lista_transacciones_vcalc_test(df_test)) 

In [17]:
# Genera columna de Precio > 600
# ==============================================================================
df_final_train['precio_mayor_600'] = df_final_train['precio_total'].apply(prec_mayor)
df_final_test['precio_mayor_600'] = df_final_test['precio_total'].apply(prec_mayor)

In [18]:
# Genera columna de Nb Max Items < 8
# ==============================================================================
df_final_train['nb_max_items_menor_8'] = df_final_train['max_num_prods_item'].apply(nb_max_menor_8)
df_final_test['nb_max_items_menor_8'] = df_final_test['max_num_prods_item'].apply(nb_max_menor_8)

In [19]:
# Generar columna con probabilidades de fraude
# ==============================================================================
df_final_train, df_final_test = generar_probf(DF_OH_TRAIN, DF_OH_TEST, 
                                        df_final_train,df_final_test)

In [20]:
# Reordenar las columnas de Train
# ==============================================================================
df_final_train = df_final_train[['id', 'num_items_dist', 'sum_items_total', 
  'precio_total', 'max_num_prods_item', 'precio_max_item', 'dif_precio_min_max',
  'precio_unit_item_mas_comprado', 'precio_mayor_600', 'nb_max_items_menor_8',
  'apple_prod', 'computers_prod', 'warranty_prod', 'fulfilment_prod', 
  'probf', 'fraud_flag']]

In [21]:
# Reordenar las columnas de Test
# ==============================================================================
df_final_test = df_final_test[['id', 'num_items_dist', 'sum_items_total', 
  'precio_total', 'max_num_prods_item', 'precio_max_item', 'dif_precio_min_max',
  'precio_unit_item_mas_comprado', 'precio_mayor_600', 'nb_max_items_menor_8',
  'apple_prod', 'computers_prod', 'warranty_prod', 'fulfilment_prod', 
  'probf']]

In [26]:
# Exportar datasets a csv
# ==============================================================================
#data_to_csv(df_final_train,"df_final_train")
#data_to_csv(df_final_test,"df_final_test")